# Multi Choice Model


*   Take the train dataset generated from the 1 way self ensemble training and train a multi choice model.
*   Use Huggingface transformers: BertForMultipleChoice
*   Will preprocess and batch the text
*   Evaluate results



### Step 1: File set up. 

In [ ]:
#Mount my drive so that I can access the split training sets. 

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#copy the dev data from colab

%cp -R /content/drive/My\ Drive/dev_SE1.csv /content/


### Step 2: Set up GPU and HuggingFace

In [ ]:
# Connect to GPU
import torch

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
GPU: Tesla P100-PCIE-16GB


In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 3.0MB 10.0MB/s 
     |████████████████████████████████| 1.1MB 30.5MB/s 
     |████████████████████████████████| 890kB 44.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=4136cfbbe5720443693e28010bd963cea1a1c58214b5803943427e92a48dc78a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### Step 3: Prep Dev Set


In [ ]:
#Imports
import pandas as pd
import numpy as np
import time
import datetime
import random
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import json
import os
from transformers import BertForMultipleChoice, AdamW, BertConfig
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split
from transformers import BertTokenizer

# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#import Dev set. 
dev_df = pd.read_csv('dev_SE1.csv')
#the empty choice is converted to a NaN when I reload, so this will correct the issue.
dev_df['a'].fillna("", inplace=True)

print('Number of dev sentences: {:,}\n'.format(dev_df.shape[0]))

Number of dev sentences: 11,873



In [ ]:
#pull out the relevant columns.
contexts = dev_df.context.values
questions = dev_df.question.values
choices = dev_df[['a','b','c','d','e']].values
#now converted to an INT
dev_df.correct_index = dev_df.correct_index.fillna(1)
labels = dev_df.correct_index.astype(int).values

In [ ]:
#token embedding loop

input_ids = []
attention_masks = []
choices_features = []

#---- THIS IS THE LOOP TO COMBINE THE QUESTIONS WITH THE CHOICES ----
for i in range(len(questions)):
    row = list(choices[i])
    temp_list = []
    for choice in row:
      text = (str(questions[i])+' '+str(choice))
      temp_list.append(text)

    encoded_dict = tokenizer(
                        [contexts[i],contexts[i],contexts[i], contexts[i], contexts[i]],
                        temp_list,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 384,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',
                        truncation = True)

# #Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# # Convert the lists into tensors.
#input_ids = torch.cat(input_ids, dim=0)
input_ids = torch.stack(input_ids)
attention_masks = torch.stack(attention_masks)
labels = torch.tensor(labels).long()

In [ ]:
#Check the shape to make sure it worked correctly. 
print(input_ids.size(0))
print(attention_masks.size(0))
print(labels.size(0))

11873
11873
11873


In [ ]:
# Set the batch size.  
batch_size = 4  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
#load the model.

model = BertForMultipleChoice.from_pretrained(
    "bert-base-cased",
    num_labels = 5,  
    output_attentions = False, 
    output_hidden_states = False)

model.load_state_dict(torch.load('/content/drive/My Drive/model_save/model_state_dict.pth'))

model.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-cased and are newly ini

BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

### Step 4: Evaluate Dev Set


In [ ]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 11,873 test sentences...
    DONE.


In [ ]:
# print(len(predictions))
# print(predictions[0])
test_list = []
# print(true_labels[0])
for i in range(5):
  for j in range(3):
    test_list.append((np.argmax(predictions[i], axis=1).flatten())[j])
#print(np.argmax(predictions[0], axis=1).flatten())
test_list
# pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

[1, 1, 1, 1, 0, 1, 4, 1, 0, 1, 1, 0, 0, 0, 1]

In [ ]:
#This will get me the indexes I need using the predictions above.

# def get_label_list(true_labels):
#   full_label_list = []
#   for i in range(len(true_labels)):
#     for j in range(len(true_labels[i])):
#       full_label_list.append(true_labels[i][j])
#   return full_label_list

def get_label_list(predictions):
  full_label_list = []
  for i in range(len(predictions)):
    for j in range(len(predictions[i])):
      full_label_list.append((np.argmax(predictions[i], axis=1).flatten())[j])
  return full_label_list

full_labels = get_label_list(predictions)

In [ ]:
#Use the functions above to get a prediction dictonary in the proper format.

def get_pred_dict(full_labels, df):
    pred_dict = {}
    for i in range(len(full_labels)):
        key = str(dev_df['id'][i])
        best_guess = str(df.iloc[i, full_labels[i]])
        pred_dict[key] = best_guess
    return pred_dict 

#Output dict into a csv.
def output_predictions(pred_dict, file_name):
    with open(file_name, 'w', encoding = 'utf-8') as json_file:
        json.dump(pred_dict, json_file, ensure_ascii=True)


In [ ]:
import json
#split out the answers into an answer dataframe to extract the correct text 
answer_df = dev_df[['a','b','c','d','e']]

pred_dict = get_pred_dict(full_labels, answer_df)
output_predictions(pred_dict, 'preds_SE1.json')


In [ ]:
# save a copy in my drive.

%cp -R /content/preds_SE1.json /content/drive/My\ Drive/model_save 

In [ ]:
# Clone SQUAD repo for the evaluation file.
# Move the eval file to my content folder 

!git clone https://github.com/white127/SQUAD-2.0-bidaf.git
%mv /content/SQUAD-2.0-bidaf/evaluate-v2.0.py /content/

Cloning into 'SQUAD-2.0-bidaf'...
remote: Enumerating objects: 125, done.
remote: Total 125 (delta 0), reused 0 (delta 0), pack-reused 125
Receiving objects: 100% (125/125), 709.51 KiB | 6.01 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
# Still download the Dev set.
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-07-19 06:02:38--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  21.3MB/s    in 0.2s    

2020-07-19 06:02:38 (21.3 MB/s) - ‘dev-v2.0.json’ saved [4370528/4370528]



In [ ]:
print("Results for SE-4, with 5 way Mutli Choice")
!python evaluate-v2.0.py dev-v2.0.json preds_SE1.json


Results for SE-4, with 5 way Mutli Choice
{
  "exact": 60.53230017687189,
  "f1": 62.96928264573781,
  "total": 11873,
  "HasAns_exact": 56.747638326585694,
  "HasAns_f1": 61.62859191174817,
  "HasAns_total": 5928,
  "NoAns_exact": 64.30613961312027,
  "NoAns_f1": 64.30613961312027,
  "NoAns_total": 5945
}


In [ ]:
##### MAKE SURE YOU MOVE A COPY TO YOUR BUCKET.

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/drive/My Drive/model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
#torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to /content/drive/My Drive/model_save/


NameError: ignored

In [ ]:
#torch.save(model, '/content/drive/My Drive/model_save/model.pt')
torch.save(model.state_dict(), '/content/drive/My Drive/model_save/model_state_dict.pth')